In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist

# Define input image dimensions
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

# Given input of noise (latent) vector, the Generator produces an image.
def build_generator():
    noise_shape = (100,)  # 1D array of size 100 (latent vector / noise)

    # Define your generator network
    model = Sequential()
    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)  # Generated image
    return Model(noise, img)

# Given an input image, the Discriminator outputs the likelihood of the image being real.
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)
def train(epochs, batch_size=128, save_interval=50):
    save1 = ''
    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3) 
    half_batch = int(batch_size / 2)
    for epoch in range(epochs):
        # ---------------------
        #  Train Discriminator
        # ---------------------
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    #take average loss from real and fake images. 
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
        # ---------------------
        #  Train Generator
        # ---------------------
        noise = np.random.normal(0, 1, (batch_size, 100)) 
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size
        g_loss = combined.train_on_batch(noise, valid_y)
        from IPython.display import clear_output
        clear_output(wait=True)
        print(save1)
        print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
        if epoch %100 ==0:
            save1 += ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]\n" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        if epoch % save_interval == 0:
            save_imgs(epoch)
            generator.save(f'models/generator_{epoch}_MNIST.h5')

# When the specific sample_interval is hit, we call the sample_image function.
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

optimizer = Adam(0.0002, 0.5)  # Learning rate and momentum.

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

z = Input(shape=(100,))
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer='adam')  

train(epochs=100000, batch_size=32, save_interval=10)

generator.save('generator_model.h5') 

0 [D loss: 0.655362, acc.: 59.38%] [G loss: 0.770850]
100 [D loss: 0.124181, acc.: 100.00%] [G loss: 5.356822]
200 [D loss: 0.061261, acc.: 100.00%] [G loss: 5.686111]
300 [D loss: 0.075949, acc.: 96.88%] [G loss: 5.143754]
400 [D loss: 0.052409, acc.: 96.88%] [G loss: 5.710277]
500 [D loss: 0.162921, acc.: 93.75%] [G loss: 7.420315]
600 [D loss: 0.098539, acc.: 100.00%] [G loss: 4.740501]
700 [D loss: 0.088939, acc.: 100.00%] [G loss: 5.161057]
800 [D loss: 0.073047, acc.: 100.00%] [G loss: 4.805999]
900 [D loss: 0.264626, acc.: 90.62%] [G loss: 5.362042]
1000 [D loss: 1.797018, acc.: 40.62%] [G loss: 2.464020]
1100 [D loss: 0.066290, acc.: 96.88%] [G loss: 4.435236]
1200 [D loss: 0.065592, acc.: 100.00%] [G loss: 4.682240]
1300 [D loss: 0.098385, acc.: 96.88%] [G loss: 5.157088]
1400 [D loss: 0.066018, acc.: 96.88%] [G loss: 4.566513]
1500 [D loss: 0.042879, acc.: 100.00%] [G loss: 4.597501]
1600 [D loss: 0.111794, acc.: 96.88%] [G loss: 3.920511]
1700 [D loss: 0.108029, acc.: 96.88%